In [ ]:
import boto3
from boto3 import session

key_id = "myaccess"
secret_key = "admin123!"
minio_url="http://minio.ml-minio.svc.cluster.local:9000"

session = boto3.session.Session(aws_access_key_id=key_id, aws_secret_access_key=secret_key)
s3_client = boto3.client("s3", aws_access_key_id=key_id, aws_secret_access_key=secret_key, endpoint_url=minio_url)

In [ ]:
! mkdir model
s3_client.download_file("model", "MLP_Keras_e19.h5", './model/model.h5')

In [ ]:
!pip install tf2onnx
!python -m tf2onnx.convert --saved-model ./model --output ./model/model.onnx --opset 13

In [11]:
s3_client.upload_file("./onnx/model.onnx", "onnx", "model.onnx")


In [ ]:
import os
import shutil
import tensorflow as tf
def save_model_to_saved_model(sess, input_tensor, output_tensor):
    from tensorflow.saved_model import simple_save
    save_path = r"./output/saved_model"
    if os.path.exists(save_path):
        shutil.rmtree(save_path)
    simple_save(sess, save_path, {input_tensor.name: input_tensor}, {output_tensor.name: output_tensor})

print("please wait for a while, because the script will train MNIST from scratch")
tf.reset_default_graph()
sess_tf, saver, input_tensor, output_tensor = create_and_train_mnist()
print("save tensorflow in format \"saved_model\"")
save_model_to_saved_model(sess_tf, input_tensor, output_tensor)